# Poisson factorization

This example notebook illustrates the usage of the [poismf](https://www.github.com/david-cortes/poismf) package for recommender systems with implicit feedback data using the [Last.FM 360k dataset](http://ocelma.net/MusicRecommendationDataset/index.html). The model is described in more detail in [Fast Non-Bayesian Poisson Factorization for Implicit-Feedback Recommendations](https://arxiv.org/abs/1811.01908).

# Model description

The basic idea is to take a sparse input matrix of counts $\mathbf{X}_{m,n}$, which in this case is given by the number of times each user (row in the matrix) played each song (column in the matrix), and find an approximation as the product of two non-negative lower-dimensional latent factor matrices $\mathbf{A}_{m,k}$ and $\mathbf{B}_{n,k}$ by maximizing Poisson likelihood, i.e. fit a model:
$$
\mathbf{X} \sim \text{Poisson}(\mathbf{A} \mathbf{B}^T)
$$

Which is then used to make predictions on the missing (zero-valued) entries, with the highest-predicted items for each user being the best candidates to recommend.

The package offers different optimization methods which have different advantages in terms of speed and quality, and depending on the settings, is usually able to find good solutions in which the latent factors matrices $\mathbf{A}$ and $\mathbf{B}$ are sparse (i.e. most entries are exactly zero).
** *
## Loading the data

In [1]:
import numpy as np, pandas as pd

lfm = pd.read_table('usersha1-artmbid-artname-plays.tsv',
                           sep='\t', header=None, names=['UserId','ItemId', 'Artist','Count'])
lfm.columns = ['UserId', 'ItemId', 'Artist', 'Count']
lfm.head(3)

,UserId,ItemId,Artist,Count
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897


In [2]:
lfm = lfm.drop('Artist', axis=1)
lfm = lfm.loc[lfm.Count > 0]
lfm['UserId'] = pd.Categorical(lfm.UserId).codes
lfm['ItemId'] = pd.Categorical(lfm.ItemId).codes
lfm.head(3)

,UserId,ItemId,Count
0,0,37425,2137
1,0,152039,1099
2,0,112365,897


In [3]:
lfm.Count.describe()

count    1.753565e+07
mean     2.151932e+02
std      6.144815e+02
min      1.000000e+00
25%      3.500000e+01
50%      9.400000e+01
75%      2.240000e+02
max      4.191570e+05
Name: Count, dtype: float64

## Producing a train-test split

This section will at first leave 30% of the data as a test set. Then, it will filter out from that test set the users and items which were not in the remaining 70% which was set as training data. This test set will be used to evaluate ranking-based recommendation metrics later.

In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(lfm, test_size=.3)
df_train = df_train.copy()
users_train = np.unique(df_train.UserId.to_numpy())
items_train = np.unique(df_train.ItemId.to_numpy())
df_test = df_test.loc[df_test.UserId.isin(users_train) &
                      df_test.ItemId.isin(items_train)]
df_train["UserId"] = pd.Categorical(df_train.UserId, users_train).codes
df_train["ItemId"] = pd.Categorical(df_train.ItemId, items_train).codes
df_test["UserId"] = pd.Categorical(df_test.UserId, users_train).codes
df_test["ItemId"] = pd.Categorical(df_test.ItemId, items_train).codes
users_test = np.unique(df_test.UserId.to_numpy())

print("Number of entries in training data: {:,}".format(df_train.shape[0]))
print("Number of entries in test data: {:,}".format(df_test.shape[0]))
print("Number of users in training data: {:,}".format(users_train.shape[0]))
print("Number of users in test data: {:,}".format(users_test.shape[0]))
print("Number of items in training and test data: {:,}".format(items_train.shape[0]))

Number of entries in training data: 12,274,957
Number of entries in test data: 5,245,386
Number of users in training data: 358,854
Number of users in test data: 358,721
Number of items in training and test data: 147,209


## Ranking metrics for evaluation

The models fit here will be evaluated by AUC and P@5, calculated for individual users and then averaged across a random sample of 1,000 users. These metrics are calculated for each user separately, by taking the entries in the hold-out test set as a positive class, entries which are neither in the training or test sets as a negative class, and producing predictions for all the entries that were not in the training set - the idea being that models which tend to rank highest the songs that the users ended up listening are better.

In [5]:
from sklearn.metrics import roc_auc_score
from joblib import Parallel, delayed

## Note: this is a computationally inefficient implementation of the
## test metrics, not recommended to use outside of this notebook
def print_ranking_metrics(A, B, df_train, df_test, users_test,
                          nusers=1000, top_n=5, seed=1,
                          njobs=-1):
    """
    Parameters
    ----------
    A : array(m, k)
        The user-factor matrix.
    B : array(n, k)
        The item-factor matrix
    df_train : DataFrame(n_train, [user, item, value])
        The training triplets.
    df_test : DataFrame(n_test, [user, item, value])
        The hold-out triplets.
    n_user : int
        Number of users to sample.
    top_n : int
        Number of top-ranked items to calculate precision.
    seed : int
        Random seed used to select the users.
    njobs : int
        Number of jobs to run in parallel.
    """
    n_users = A.shape[0]
    n_items = B.shape[0]
    rng = np.random.default_rng(seed=seed)
    chosen_users = rng.choice(users_test, size=nusers, replace=False)
    all_train = df_train.loc[df_train.UserId.isin(chosen_users)]
    all_test = df_test.loc[df_test.UserId.isin(chosen_users)]
    
    def metrics_single_user(user):
        ypos = all_test.ItemId.loc[all_test.UserId == user].to_numpy()
        ytrain = all_train.ItemId.loc[all_train.UserId == user].to_numpy()
        yneg = np.setdiff1d(np.arange(n_items), np.r_[ypos, ytrain])
        ytest = np.r_[yneg, ypos]
        yhat = B[ytest].dot(A[user])
        auc = roc_auc_score(np.r_[np.zeros(yneg.shape[0]),
                                  np.ones(ypos.shape[0])],
                            yhat)
        topN = np.argsort(-yhat)[:top_n]
        p_at_k = np.mean(topN >= yneg.shape[0])
        p_at_k_rnd = ypos.shape[0] / ytest.shape[0] ## <- baseline
        return auc, p_at_k, p_at_k_rnd

    res_triplets = Parallel(n_jobs = njobs)\
                    (delayed(metrics_single_user)(u) \
                        for u in chosen_users)

    res_triplets = np.array(res_triplets)
    auc = np.mean(res_triplets[:,0])
    p_at_k = np.mean(res_triplets[:,1])
    p_at_k_rnd = np.mean(res_triplets[:,2])
    print("AUC: %.4f [random: %.2f]" % (auc, 0.5))
    print("P@%d: %.4f [random: %.4f]" % (top_n,
                                         p_at_k,
                                         p_at_k_rnd))

## Fitting the model
** *
This section will fit and evaluate the Poisson factorization model fit with different hyperparameters.

In [6]:
from poismf import PoisMF

Oriented towards speed:

In [7]:
%%time
model_fast = PoisMF(reindex=False, method="pg",
                    k=10, niter=10, maxupd=1, l2_reg=1e9)\
                .fit(df_train)

CPU times: user 1min 13s, sys: 140 ms, total: 1min 13s
Wall time: 6.27 s


In [8]:
print_ranking_metrics(model_fast.A, model_fast.B,
                      df_train, df_test, users_test)

AUC: 0.9505 [random: 0.50]
P@5: 0.0948 [random: 0.0001]


Faster, but still not-so-good quality:

In [9]:
%%time
model_balanced = PoisMF(reindex=False, method="cg",
                        k=50, niter=25, maxupd=5, l2_reg=1e5)\
                    .fit(df_train)

CPU times: user 39min 42s, sys: 1.31 s, total: 39min 43s
Wall time: 2min 32s


In [10]:
print_ranking_metrics(model_balanced.A, model_balanced.B,
                      df_train, df_test, users_test)

AUC: 0.9685 [random: 0.50]
P@5: 0.1048 [random: 0.0001]


Good quality and producing sparse factors, but slow:

In [11]:
%%time
## Note: 'maxupd' for 'tncg' means 'maxfneval'
model_good = PoisMF(reindex=False, method="tncg",
                    k=100, niter=50, maxupd=750, l2_reg=1e3)\
                .fit(df_train)

CPU times: user 6h 47min 51s, sys: 7.06 s, total: 6h 47min 58s
Wall time: 25min 44s


In [12]:
print_ranking_metrics(model_good.A, model_good.B,
                      df_train, df_test, users_test)

AUC: 0.9584 [random: 0.50]
P@5: 0.1552 [random: 0.0001]


(In this case, it's possible to increase P@5 at the expense of AUC by decreasing  the regulatization parameter)
** *
### Sparse factors

Verifying that the obtain latent factors are indeed sparse:

In [13]:
model_good.A[0]

array([0.        , 0.        , 0.23730212, 0.13069925, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.36238268, 0.        , 0.10536244, 0.        ,
       0.0592634 , 0.15726347, 0.        , 0.        , 0.        ,
       0.        , 0.11451453, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.19900147,
       0.        , 0.18398437, 0.        , 0.        , 0.51381418,
       0.        , 0.        , 1.18098522, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.18386942, 0.        ,
       0.08473978, 0.13078214, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.10739615, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.36411405, 0.     

In [14]:
print("Percent of zero-valued entries in A: %.2f%%" %
      float((model_good.A == 0.).mean() * 100.))

Percent of zero-valued entries in A: 90.11%


In [15]:
print("Percent of zero-valued entries in B: %.2f%%" %
      float((model_good.B == 0.).mean() * 100.))

Percent of zero-valued entries in B: 94.01%


## Prediction API
** *
Ranking top-N items IDs for a given user:

In [16]:
model_good.topN(user = 2, n = 5,
                exclude = df_train.ItemId.loc[df_train.UserId==2])

array([101879,  76011,      0,  40417, 117964], dtype=uint64)

(These numbers correspond to the IDs of the items in the data that was passed)

If it were a new user - note that the obtained latent factors will differ slightly and it might affect the ranking:

In [17]:
model_good.topN_new(df_train.loc[df_train.UserId==2], n = 5,
                     exclude = df_train.ItemId.loc[df_train.UserId==2])

array([101879,  96245,      0,  76011, 105862], dtype=uint64)

Predicting new (user,item) combinations:

In [18]:
### Predicts triplets (3,4), (3,5), (10,11)
model_good.predict(user=[3,3,3], item=[3,4,11])

array([0.00000000e+00, 2.75115452e-18, 1.03921970e-17])

Obtaining latent factors for new data:

In [19]:
model_good.predict_factors(df_train.loc[df_train.UserId==2])

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00100564, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.32184424,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.25628843, 0.        , 0.        , 0.        , 0.        ,
       0.03473658, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04203724, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.08095045, 0.        , 0.        , 0.     

## Comparison against other factorization models
** *

In [20]:
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from hpfrec import HPF ### <- Bayesian version
from cmfrec import MostPopular

## Note: package implicit takes a matrix of shape [items, users]
## Other packages take a matrix of shape [users, items]
Xcoo = coo_matrix((df_train.Count, (df_train.UserId, df_train.ItemId)))
Xcoo_T = Xcoo.T
Xcsr_T = csr_matrix(Xcoo_T)

In [21]:
%%time
non_personalized = MostPopular(implicit=True).fit(Xcoo)

CPU times: user 137 ms, sys: 28 ms, total: 165 ms
Wall time: 165 ms


In [22]:
print_ranking_metrics(np.ones((Xcoo.shape[0],1)),
                      non_personalized.item_bias_.reshape((-1,1)),
                      df_train, df_test, users_test)

AUC: 0.9506 [random: 0.50]
P@5: 0.0948 [random: 0.0001]


In [23]:
%%time
ials = AlternatingLeastSquares(factors=100, regularization=0.01,
                               dtype=np.float64, iterations=15)
ials.fit(Xcsr_T)


CPU times: user 10min 50s, sys: 1.89 s, total: 10min 51s
Wall time: 46.7 s


In [24]:
print_ranking_metrics(ials.user_factors, ials.item_factors,
                      df_train, df_test, users_test)

AUC: 0.9683 [random: 0.50]
P@5: 0.2026 [random: 0.0001]


In [25]:
%%time
bpr = BayesianPersonalizedRanking(factors=100, learning_rate=0.01,
                                  regularization=0.01, dtype=np.float64,
                                  iterations=100)
bpr.fit(Xcoo_T)


CPU times: user 1h 4min 22s, sys: 876 ms, total: 1h 4min 23s
Wall time: 4min 7s


In [26]:
print_ranking_metrics(bpr.user_factors, bpr.item_factors,
                      df_train, df_test, users_test)

AUC: 0.9495 [random: 0.50]
P@5: 0.0852 [random: 0.0001]


In [27]:
%%time
hpf = HPF(k=50, verbose=False).fit(Xcoo)

CPU times: user 24min 48s, sys: 1.57 s, total: 24min 49s
Wall time: 2min 10s


In [28]:
print_ranking_metrics(hpf.Theta, hpf.Beta,
                      df_train, df_test, users_test)

AUC: 0.9713 [random: 0.50]
P@5: 0.1120 [random: 0.0001]


In [29]:
%%time
hpf = HPF(verbose=False, k=100, maxiter=150, stop_crit="maxiter").fit(Xcoo)

CPU times: user 5h 29min 3s, sys: 16.5 s, total: 5h 29min 19s
Wall time: 27min 21s


In [30]:
print_ranking_metrics(hpf.Theta, hpf.Beta,
                      df_train, df_test, users_test)

AUC: 0.9744 [random: 0.50]
P@5: 0.1284 [random: 0.0001]


** *
# References

* Cortes, David. "Fast Non-Bayesian Poisson Factorization for Implicit-Feedback Recommendations." arXiv preprint arXiv:1811.01908 (2018).